In [1]:
import requests
from requests.exceptions import ReadTimeout, ConnectTimeout
import math
import pandas as pd
# 关于地图坐标系：
# WGS84：为一种大地坐标系，也是目前广泛使用的GPS全球卫星定位系统使用的坐标系。
# GCJ02：又称火星坐标系，是由中国国家测绘局制定的地理坐标系统，是由WGS84加密后得到的坐标系。
# BD09：为百度坐标系，在GCJ02坐标系基础上再次加密。其中bd09ll表示百度经纬度坐标，bd09mc表示百度墨卡托米制坐标。
# 使用百度地图的服务，需使用BD09坐标。

In [2]:
# 读取百度WebAPI用户AK
def read_AK():
    with open('./Baidu_AK.txt','r') as f:
        AK_str = f.read()
    return AK_str

# 结构化地址转换成经纬度
def transform(geo):
    parameters = {"address": geo,
                  "ak":  read_AK(),
                  "output": 'json'}
    base =  "http://api.map.baidu.com/geocoding/v3/"    
    try:
        response = requests.get(base, parameters, timeout=2)
        if response.status_code == 200:
            answer = response.json()
            result = answer['result']
            location = result['location']
            lat = location.get('lat')
            lon = location.get('lng')
            loc = str(lat)+','+str(lon)
        else:
            pass
    except (ReadTimeout, ConnectTimeout):
        pass
    return loc

# 圆形区域检索周边poi
def search_around(proj_name, radi):
    # 默认搜索类型，为
    # query =“房地产:住宅区”
    # 需要严格按照百度的POI分类搜索，只设置query值，不设置tag值
    # query值的格式： "一级分类：二级分类"
    base = "http://api.map.baidu.com/place/v2/search"
    parameters ={'query' : '房地产:住宅区',
                 'location' : transform(proj_name),
                 'radius' : radi,
                 'output': 'json', 
                 'ak' :  read_AK(),
                 'page_size' : 20,
                 'scope': 2
                }
    try:
        response = requests.get(base, parameters, timeout=2)
        if response.status_code == 200:
            answer = response.json()
        else:
            pass
    except (ReadTimeout, ConnectTimeout):
        pass
    POIs_num = answer['total']
    # 把所有页的POI全部抓取下来
    pages_num = math.ceil(int(POIs_num) / 20)
    results = pd.DataFrame(columns=['UID','百度名称', '坐标', '距离', '地址'])
    for i in range(pages_num):
        parameters.update({ 'page_num' : i })
        response = requests.get(base, parameters)
        answer = response.json()
        # 将信息存入DataFrame
        for j in answer['results'][:]:
            loc_as_list = list(j.get('location').values())
            info_dict={'UID' :  j.get('uid'),
                       '百度名称': j.get('name'),
                       '坐标':     "{},{}".format(loc_as_list[0],loc_as_list[1]),
                       '距离':     j.get('detail_info').get('distance'), 
                       '地址':     j.get('address')}
            results = results.append(info_dict,ignore_index=True)
            
    print("{}(坐标[{}])周围{}米内供检索到{}个住宅小区信息。".format(proj_name,parameters['location'],radi,POIs_num))
    return results

In [3]:
s = search_around('上海市奉贤区传悦坊',3000)
s.to_csv('./Bmap_info.csv', encoding='utf_8_sig')
s.head()

上海市奉贤区传悦坊(坐标[30.924129963189593,121.49303258303983])周围3000米内供检索到82个住宅小区信息。


,UID,百度名称,坐标,距离,地址
0,6c6edeb4738525803904b162,古华新村B区,"30.924273,121.473137",1897,上海市奉贤区古华园路与新建中路交叉路口往东南约100米(古华新村)
1,31dcddda26d1e02261685815,金昊雅苑-北区,"30.923858,121.509635",1583,上海市奉贤区光昊路218弄
2,5c44ef97d13742ada4c75688,绿地上海之渔,"30.940928,121.497638",1918,上海市奉贤区Ｙ３６８与航南公路交叉路口东南侧(合景万景峰)
3,35debf29c1981a45a6da390a,景河苑,"30.941035,121.46435",3319,上海市奉贤区南桥镇八字桥路599弄
4,8766076b8dceaba3223bf267,花轩房产(联运大厦居民住宅区北),"30.920715,121.475465",1718,上海市奉贤区南桥环城东路与南桥曙光路交叉路口南侧(曙光新村)
